In [2]:
import uuid   # Unique identifier
import os
import time
import cv2
from PIL import Image

In [13]:
def extract_equally_spaced_frames(video_path, output_folder,status_id, person_id, num_frames=21,):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    skip_frames = int(frame_rate * 10)  # Skip the first 10 seconds
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for i in range(num_frames):
        frame_idx = skip_frames + (frame_count - skip_frames) * i // num_frames
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        
        if ret:
            output_path = os.path.join(output_folder, status_id+'_'+person_id+'_'+str(uuid.uuid1())+'.jpg')
            cv2.imwrite(output_path, frame)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    
    cap.release()

In [26]:
status_num = ['0','5','10']
status_dict ={
    '0': 'alert',
    '5': 'normal',
    '10': 'drowsy'
}
for video in range(46,47):
    for status in status_num:
        video_path = os.path.join('raw_data','videos', str(video), status+'.m4v')
        IMAGES_PATH = os.path.join('raw_data', 'images',status_dict[status])
        extract_equally_spaced_frames(video_path, output_folder=IMAGES_PATH, num_frames=21, status_id=status_dict[status], person_id=str(video))

OpenCV: Couldn't read video stream from file "raw_data/46/10.m4v"


In [12]:
def resize_and_fill(input_path, output_path, size, fill_color=(0, 0, 0)):
    # Open the image
    image = Image.open(input_path)
    # Calculate the aspect ratio of the image
    aspect_ratio = image.width / image.height
    # Determine the new dimensions while maintaining the aspect ratio
    if aspect_ratio > 1:
        new_width = size
        new_height = int(size / aspect_ratio)
    else:
        new_width = int(size * aspect_ratio)
        new_height = size
    # Resize the image while maintaining the aspect ratio
    resized_image = image.resize((new_width, new_height), resample=Image.BILINEAR)
    # Create a new image with the specified background color
    new_image = Image.new("RGB", (size, size), fill_color)
    # Calculate the position to paste the resized image
    paste_position = ((size - new_width) // 2, (size - new_height) // 2)
    # Paste the resized image onto the new image
    new_image.paste(resized_image, paste_position)
    # Save the final image. Quality=100 because they are already compressed images from taking screenshots.
    new_image.save(output_path, format="JPEG", quality=100)



In [13]:
# Desired size of the square
desired_size = 640
# Fill color (black in this example)
fill_color = (0, 0, 0)

datasets = ['train', 'test', 'val']

parent_directory = os.path.dirname(os.path.abspath(''))
for dataset in datasets:

    #Input directory
    input_directory = os.path.join(parent_directory,'raw_data', 'raw_images',dataset)
    # Output directory for resized images
    output_directory = os.path.join(parent_directory,'raw_data', 'processed_images',dataset)
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    # Loop through each image file in the input directory
    for file_name in os.listdir(input_directory):
        if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            input_path = os.path.join(input_directory, file_name)
            output_filename = os.path.splitext(file_name)[0] + "_sq.jpg"
            output_path = os.path.join(output_directory, output_filename)
            # Resize and fill the image, then save it to the output directory
            resize_and_fill(input_path, output_path, desired_size, fill_color)
    print("Image resizing and filling complete.")

Image resizing and filling complete.
Image resizing and filling complete.
Image resizing and filling complete.


In [13]:
def crop_to_square(image_path,output_path, target_size=640):
    # Open the image
    image = Image.open(image_path)
    
    # Get the original width and height of the image
    width, height = image.size
    
    # Calculate the cropping coordinates to keep the center part
    new_width = new_height = min(width, height)
    left = (width - new_width) // 2
    top = (height - new_height) // 2
    right = left + new_width
    bottom = top + new_height
    
    # Crop the image to the calculated coordinates
    cropped_image = image.crop((left, top, right, bottom))
    # Resize the cropped image to the target size
    resized_image = cropped_image.resize((target_size, target_size), Image.BILINEAR)
    resized_image.save(output_path, format="JPEG", quality=100)
  




In [14]:
# Example usage
parent_directory = os.path.dirname(os.path.abspath(''))
input_image_path = os.path.join(parent_directory,'raw_data', '23')
output_directory = os.path.join(parent_directory,'raw_data', 'processed_23',)


for file_name in os.listdir(input_image_path):
        if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            input_path = os.path.join(input_image_path, file_name)
            output_filename = os.path.splitext(file_name)[0] + "_sq.jpg"
            output_path = os.path.join(output_directory, output_filename)
            # Resize and fill the image, then save it to the output directory
            crop_to_square(input_path,output_path, target_size = 640)